# Preprocessing and Training Data

In [2]:
# import necessary packages and load dataset

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn import tree
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import confusion_matrix, classification_report
from tabulate import tabulate


In [3]:
df = pd.read_csv('../data/hotel_data_cleaned.csv')

In [4]:
df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day,stays_in_weekend_nights,stays_in_week_nights,adults,...,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date_year,reservation_status_date_month,reservation_status_date_day,reservation_status_day_of_week,arrival_date_day_of_week
0,Resort Hotel,0,7,2015,7,27,1,0,1,1,...,Transient,75.0,0,0,Check-Out,2015,7,2,Thursday,Wednesday
1,Resort Hotel,0,13,2015,7,27,1,0,1,1,...,Transient,75.0,0,0,Check-Out,2015,7,2,Thursday,Wednesday
2,Resort Hotel,0,14,2015,7,27,1,0,2,2,...,Transient,98.0,0,1,Check-Out,2015,7,3,Friday,Wednesday
3,Resort Hotel,0,14,2015,7,27,1,0,2,2,...,Transient,98.0,0,1,Check-Out,2015,7,3,Friday,Wednesday
4,Resort Hotel,0,0,2015,7,27,1,0,2,2,...,Transient,107.0,0,0,Check-Out,2015,7,3,Friday,Wednesday


In [5]:
df.dtypes

hotel                              object
is_canceled                         int64
lead_time                           int64
arrival_date_year                   int64
arrival_date_month                  int64
arrival_date_week_number            int64
arrival_date_day                    int64
stays_in_weekend_nights             int64
stays_in_week_nights                int64
adults                              int64
children                          float64
babies                              int64
meal                               object
country                            object
market_segment                     object
distribution_channel               object
is_repeated_guest                   int64
previous_cancellations              int64
previous_bookings_not_canceled      int64
reserved_room_type                 object
assigned_room_type                 object
booking_changes                     int64
deposit_type                       object
agent                             

In [6]:
# dropping target leakage variable

df=df.drop(columns='reservation_status')
df

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day,stays_in_weekend_nights,stays_in_week_nights,adults,...,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status_date_year,reservation_status_date_month,reservation_status_date_day,reservation_status_day_of_week,arrival_date_day_of_week
0,Resort Hotel,0,7,2015,7,27,1,0,1,1,...,0,Transient,75.00,0,0,2015,7,2,Thursday,Wednesday
1,Resort Hotel,0,13,2015,7,27,1,0,1,1,...,0,Transient,75.00,0,0,2015,7,2,Thursday,Wednesday
2,Resort Hotel,0,14,2015,7,27,1,0,2,2,...,0,Transient,98.00,0,1,2015,7,3,Friday,Wednesday
3,Resort Hotel,0,14,2015,7,27,1,0,2,2,...,0,Transient,98.00,0,1,2015,7,3,Friday,Wednesday
4,Resort Hotel,0,0,2015,7,27,1,0,2,2,...,0,Transient,107.00,0,0,2015,7,3,Friday,Wednesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116946,City Hotel,0,23,2017,8,35,30,2,5,2,...,0,Transient,96.14,0,0,2017,9,6,Wednesday,Wednesday
116947,City Hotel,0,102,2017,8,35,31,2,5,3,...,0,Transient,225.43,0,2,2017,9,7,Thursday,Thursday
116948,City Hotel,0,34,2017,8,35,31,2,5,2,...,0,Transient,157.71,0,4,2017,9,7,Thursday,Thursday
116949,City Hotel,0,109,2017,8,35,31,2,5,2,...,0,Transient,104.40,0,0,2017,9,7,Thursday,Thursday


## Transform categorical, text data

In [7]:
df1= pd.get_dummies(df, columns=['hotel','arrival_date_day_of_week','meal', 'country', 
                                 'market_segment', 'distribution_channel', 'reserved_room_type', 
                                 'assigned_room_type', 'deposit_type', 'customer_type',
                                 'reservation_status_day_of_week'], drop_first=False)
df1.shape

(116951, 260)

In [8]:
df.agent.value_counts()

9.0      31813
0.0      15231
240.0    13763
1.0       7050
14.0      3614
         ...  
352.0        1
480.0        1
451.0        1
165.0        1
472.0        1
Name: agent, Length: 332, dtype: int64

## Train/Test Split (Churn)

In [9]:
len(df1) * .7, len(df1) * .3

(81865.7, 35085.299999999996)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(df1.drop(columns='is_canceled'), 
                                                    df1.is_canceled, test_size=0.3, 
                                                    random_state=100)

In [11]:
X_train.shape, X_test.shape

((81865, 259), (35086, 259))

In [12]:
y_train.shape, y_test.shape

((81865,), (35086,))

# Classification Models 

1. Logistic Regression 
2. KNN 
3. SVM
4. Decision Tree
5. Random Forest
6. Gradient Boosting


### Logistic Regression

In [12]:
logreg= LogisticRegression()

logreg.fit(X_train, y_train)

/Users/soyoungan/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [13]:
prediction_log = logreg.predict(X_test)

In [14]:
print (confusion_matrix(y_test, prediction_log))
print (classification_report(y_test, prediction_log))

[[19966  1919]
 [ 8205  4996]]
              precision    recall  f1-score   support

           0       0.71      0.91      0.80     21885
           1       0.72      0.38      0.50     13201

    accuracy                           0.71     35086
   macro avg       0.72      0.65      0.65     35086
weighted avg       0.71      0.71      0.68     35086



Note for Logistic Regression

0 Precision represents the percent of your no-cancelled classifier 
1 Precision represents  cancelled prediction 

### KNN

In [15]:
# neighbors = np.arange(3, 5)
# train_accuracy = np.empty(len(neighbors))
# test_accuracy = np.empty(len(neighbors))

# for i, k in enumerate (neighbors):
#     print (k)
#     knn = KNeighborsClassifier(n_neighbors=k)
#     knn.fit(X_train, y_train)
#     train_accuracy[i]=knn.score(X_train, y_train)
#     test_accuracy[i]=knn.score(X_test, y_test)

# plt.title('KNN: Varying Number of Neighbors')
# plt.plot(neighbors, test_accuracy, label='Testing Accuracy')
# plt.plot(neighbors, train_accuracy, label='Training Accuracy')
# plt.legend()
# plt.xlabel('Number of Neighbors')
# plt.ylabel('Accuracy')
# plt.show()


In [16]:
clf_knn = KNeighborsClassifier(n_neighbors=5)

clf_knn.fit(X_train, y_train)
y_pred_knn = clf_knn.predict(X_test)

In [17]:
print (confusion_matrix(y_test, y_pred_knn))
print (classification_report(y_test, y_pred_knn))

[[20633  1252]
 [ 3414  9787]]
              precision    recall  f1-score   support

           0       0.86      0.94      0.90     21885
           1       0.89      0.74      0.81     13201

    accuracy                           0.87     35086
   macro avg       0.87      0.84      0.85     35086
weighted avg       0.87      0.87      0.86     35086



### Linear SVM

In [18]:
clf_lsvm = LinearSVC()

clf_lsvm.fit(X_train, y_train)
y_pred_svm = clf_lsvm.predict(X_test)


/Users/soyoungan/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [19]:
print (confusion_matrix(y_test, y_pred_svm))
print (classification_report(y_test, y_pred_svm))

[[ 6645 15240]
 [  305 12896]]
              precision    recall  f1-score   support

           0       0.96      0.30      0.46     21885
           1       0.46      0.98      0.62     13201

    accuracy                           0.56     35086
   macro avg       0.71      0.64      0.54     35086
weighted avg       0.77      0.56      0.52     35086



### Decision Tree

In [13]:
# default params: gini, max_depth = None, min_samples_split = 2, min_samples_leaf = 1

clf_dt = tree.DecisionTreeClassifier(random_state = 47)

clf_dt.fit(X_train, y_train)
y_pred_dt = clf_dt.predict(X_test)

In [14]:
print (confusion_matrix(y_test, y_pred_dt))
print (classification_report(y_test, y_pred_dt))

[[21089   796]
 [  893 12308]]
              precision    recall  f1-score   support

           0       0.96      0.96      0.96     21885
           1       0.94      0.93      0.94     13201

    accuracy                           0.95     35086
   macro avg       0.95      0.95      0.95     35086
weighted avg       0.95      0.95      0.95     35086



#### Feature Importance - Decision Tree

In [15]:
headers = ["name", "score"]
values_dt = sorted(zip(X_train.columns, clf_dt.feature_importances_), key=lambda x: x[1] * -1)
print('Feature Importance - Decision Tree')
print(tabulate(values_dt, headers, tablefmt="plain"))

Feature Importance - Decision Tree
name                                            score
deposit_type_Non Refund                   0.231992
reservation_status_date_month             0.081724
arrival_date_week_number                  0.0791298
market_segment_Online TA                  0.0623794
lead_time                                 0.0537492
total_of_special_requests                 0.0481218
reservation_status_date_day               0.047269
country_PRT                               0.0410268
arrival_date_month                        0.0355712
agent                                     0.0332353
reservation_status_date_year              0.0289941
arrival_date_day                          0.0283431
arrival_date_year                         0.0229983
required_car_parking_spaces               0.022191
reservation_status_day_of_week_Sunday     0.0205965
stays_in_week_nights                      0.0169461
adr                                       0.0156025
previous_cancellations         

In [16]:
top50_feature_dt = [item[0] for item in values_dt[:50]]

In [17]:
X_train_dt=X_train[top50_feature_dt]
X_test_dt=X_test[top50_feature_dt]

In [18]:
clf_dt.fit(X_train_dt, y_train)

DecisionTreeClassifier(random_state=47)

In [19]:
y_pred3 = clf_dt.predict(X_test_dt)

In [20]:
print (confusion_matrix(y_test, y_pred3))
print (classification_report(y_test, y_pred3))

[[21061   824]
 [  845 12356]]
              precision    recall  f1-score   support

           0       0.96      0.96      0.96     21885
           1       0.94      0.94      0.94     13201

    accuracy                           0.95     35086
   macro avg       0.95      0.95      0.95     35086
weighted avg       0.95      0.95      0.95     35086



In [21]:
top100_feature_dt = [item[0] for item in values_dt[:100]]
X_train_dt=X_train[top100_feature_dt]
X_test_dt=X_test[top100_feature_dt]

In [22]:
clf_dt.fit(X_train_dt, y_train)
y_pred3 = clf_dt.predict(X_test_dt)

In [23]:
print (confusion_matrix(y_test, y_pred3))
print (classification_report(y_test, y_pred3))

[[21056   829]
 [  897 12304]]
              precision    recall  f1-score   support

           0       0.96      0.96      0.96     21885
           1       0.94      0.93      0.93     13201

    accuracy                           0.95     35086
   macro avg       0.95      0.95      0.95     35086
weighted avg       0.95      0.95      0.95     35086



In [24]:
top25_feature_dt = [item[0] for item in values_dt[:25]]
X_train_dt=X_train[top25_feature_dt]
X_test_dt=X_test[top25_feature_dt]

In [25]:
clf_dt.fit(X_train_dt, y_train)
y_pred3 = clf_dt.predict(X_test_dt)

In [26]:
print (confusion_matrix(y_test, y_pred3))
print (classification_report(y_test, y_pred3))

[[21047   838]
 [  884 12317]]
              precision    recall  f1-score   support

           0       0.96      0.96      0.96     21885
           1       0.94      0.93      0.93     13201

    accuracy                           0.95     35086
   macro avg       0.95      0.95      0.95     35086
weighted avg       0.95      0.95      0.95     35086



### Gradient Boosting

In [52]:
clf_gb = GradientBoostingClassifier(random_state = 47)

clf_gb.fit(X_train, y_train)
y_pred_gb = clf_gb.predict(X_test)

In [53]:
print (confusion_matrix(y_test, y_pred_gb))
print (classification_report(y_test, y_pred_gb))

[[20962   923]
 [ 2554 10647]]
              precision    recall  f1-score   support

           0       0.89      0.96      0.92     21885
           1       0.92      0.81      0.86     13201

    accuracy                           0.90     35086
   macro avg       0.91      0.88      0.89     35086
weighted avg       0.90      0.90      0.90     35086



#### Feature Importance - Gradient Boosting

In [54]:
headers = ["name", "score"]
values_gb = sorted(zip(X_train.columns, clf_gb.feature_importances_), key=lambda x: x[1] * -1)
print('Feature Importance - Gradient Boosting')
print(tabulate(values_gb, headers, tablefmt="plain"))

Feature Importance - Gradient Boosting
name                                            score
deposit_type_Non Refund                   0.340067
country_PRT                               0.0925429
lead_time                                 0.0819307
market_segment_Online TA                  0.0736457
total_of_special_requests                 0.0588768
reservation_status_date_month             0.0367448
arrival_date_month                        0.0328232
arrival_date_week_number                  0.0306488
required_car_parking_spaces               0.0304181
previous_cancellations                    0.0277206
reservation_status_day_of_week_Sunday     0.0268049
stays_in_week_nights                      0.0241976
stays_in_weekend_nights                   0.0226044
agent                                     0.0173813
reservation_status_date_year              0.0170523
booking_changes                           0.015071
customer_type_Transient                   0.0121362
arrival_date_year        

### Random Forest

In [27]:
rf = RandomForestClassifier(random_state=47)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

In [28]:
print (confusion_matrix(y_test, y_pred_rf))
print (classification_report(y_test, y_pred_rf))

[[21654   231]
 [ 1461 11740]]
              precision    recall  f1-score   support

           0       0.94      0.99      0.96     21885
           1       0.98      0.89      0.93     13201

    accuracy                           0.95     35086
   macro avg       0.96      0.94      0.95     35086
weighted avg       0.95      0.95      0.95     35086



#### Feature Importance - RF

In [29]:
headers = ["name", "score"]
values_rf = sorted(zip(X_train.columns, rf.feature_importances_), key=lambda x: x[1] * -1)
print('Feature Importance - Random Forest')
print(tabulate(values_rf, headers, tablefmt="plain"))

Feature Importance - Random Forest
name                                            score
lead_time                                 0.0692112
deposit_type_Non Refund                   0.062218
country_PRT                               0.0586881
reservation_status_date_month             0.0577651
deposit_type_No Deposit                   0.049818
total_of_special_requests                 0.0440357
arrival_date_week_number                  0.0432837
adr                                       0.0396664
reservation_status_date_day               0.0368257
arrival_date_month                        0.0338692
arrival_date_day                          0.0328172
agent                                     0.030052
stays_in_week_nights                      0.0272971
stays_in_weekend_nights                   0.0235854
previous_cancellations                    0.0225817
reservation_status_date_year              0.017155
required_car_parking_spaces               0.0167015
market_segment_Online TA       

In [30]:
# top 50 features 

top50_feature_rf = [item[0] for item in values_rf[:50]]

In [31]:
X_train_rf=X_train[top50_feature_rf]
X_test_rf=X_test [top50_feature_rf]

In [32]:
rf.fit(X_train_rf, y_train)

RandomForestClassifier(random_state=47)

In [33]:
y_pred2 = rf.predict(X_test_rf)

In [34]:
print (confusion_matrix(y_test, y_pred2))
print (classification_report(y_test, y_pred2))

[[21646   239]
 [ 1155 12046]]
              precision    recall  f1-score   support

           0       0.95      0.99      0.97     21885
           1       0.98      0.91      0.95     13201

    accuracy                           0.96     35086
   macro avg       0.96      0.95      0.96     35086
weighted avg       0.96      0.96      0.96     35086



In [35]:
# top 100 features

top100_feature_rf = [item[0] for item in values_rf[:100]]

In [36]:
X_train_rf=X_train[top100_feature_rf]
X_test_rf=X_test [top100_feature_rf]

In [37]:
rf.fit(X_train_rf, y_train)

RandomForestClassifier(random_state=47)

In [38]:
y_pred2 = rf.predict(X_test_rf)

In [39]:
# top 100 features

print (confusion_matrix(y_test, y_pred2))
print (classification_report(y_test, y_pred2))

[[21629   256]
 [ 1319 11882]]
              precision    recall  f1-score   support

           0       0.94      0.99      0.96     21885
           1       0.98      0.90      0.94     13201

    accuracy                           0.96     35086
   macro avg       0.96      0.94      0.95     35086
weighted avg       0.96      0.96      0.95     35086



In [40]:
# top 20

top20_feature_rf = [item[0] for item in values_rf[:20]]

In [42]:
X_train_rf=X_train[top20_feature_rf]
X_test_rf=X_test [top20_feature_rf]

In [43]:
rf.fit(X_train_rf, y_train)
y_pred2 = rf.predict(X_test_rf)

In [44]:
# top 20 features

print (confusion_matrix(y_test, y_pred2))
print (classification_report(y_test, y_pred2))

[[21717   168]
 [  951 12250]]
              precision    recall  f1-score   support

           0       0.96      0.99      0.97     21885
           1       0.99      0.93      0.96     13201

    accuracy                           0.97     35086
   macro avg       0.97      0.96      0.97     35086
weighted avg       0.97      0.97      0.97     35086



### Refine the Decision Tree Model (Tuning Parameters)

In [74]:
param_grid_dt ={
    "criterion":['gini','entropy'],
    "max_depth":range(70, 200),
    "min_samples_split":range(1,10),
    "min_samples_leaf":range(1,10)
}

In [75]:
# Randomized Search

random_dt = RandomizedSearchCV(clf_dt, param_distributions = param_grid_dt, cv=5, verbose=1, n_jobs=-1, n_iter=200)

random_dt.fit(X_train, y_train)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  8.1min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed: 14.8min
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed: 19.6min finished


RandomizedSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=47),
                   n_iter=200, n_jobs=-1,
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': range(70, 200),
                                        'min_samples_leaf': range(1, 10),
                                        'min_samples_split': range(1, 10)},
                   verbose=1)

In [76]:
random_dt.best_params_

{'min_samples_split': 6,
 'min_samples_leaf': 2,
 'max_depth': 135,
 'criterion': 'entropy'}

In [77]:
### Grid Search

In [78]:
param_grid ={
    "criterion":['gini','entropy'],
    "max_depth":range(140, 175),
    "min_samples_split":range(3,8),
    "min_samples_leaf":range(1,5)
}

In [79]:
grid_dt = GridSearchCV(clf_dt, param_grid = param_grid, cv=5, verbose=1, n_jobs=-1)

grid_dt.fit(X_train, y_train)

Fitting 5 folds for each of 1400 candidates, totalling 7000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 10.3min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed: 18.8min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed: 27.6min
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed: 101.1min
[Parallel(n_jobs=-1)]: Done 2442 tasks      | elapsed: 118.8min
[Parallel(n_jobs=-1)]: Done 3192 tasks      | elapsed: 135.8min
[Parallel(n_jobs=-1)]: Done 4042 tasks      | elapsed: 153.8min
[Parallel(n_jobs=-1)]: Done 4992 tasks      | elapsed: 172.5min
[Parallel(n_jobs=-1)]: Done 6042 tasks      | elapsed: 194.3min
[Parallel(n_jobs=-1)]: Done 7000 out of 7000 | elapsed: 215.7min finished


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=47), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(140, 175),
                         'min_samples_leaf': range(1, 5),
                         'min_samples_split': range(3, 8)},
             verbose=1)

In [80]:
grid_dt.best_params_

{'criterion': 'entropy',
 'max_depth': 140,
 'min_samples_leaf': 1,
 'min_samples_split': 3}

#### implementing optimized parameters to Decision Tree Model


In [45]:
clf_dt2 = tree.DecisionTreeClassifier(criterion='entropy',max_depth=140, 
                                     min_samples_leaf=1, min_samples_split=3, random_state=47)

clf_dt2.fit(X_train, y_train)
y_pred_dt2 = clf_dt2.predict(X_test)

In [46]:
print (confusion_matrix(y_test, y_pred_dt2))
print (classification_report(y_test, y_pred_dt2))

[[21166   719]
 [  823 12378]]
              precision    recall  f1-score   support

           0       0.96      0.97      0.96     21885
           1       0.95      0.94      0.94     13201

    accuracy                           0.96     35086
   macro avg       0.95      0.95      0.95     35086
weighted avg       0.96      0.96      0.96     35086



In [47]:
headers = ["name", "score"]
values_dt = sorted(zip(X_train.columns, clf_dt2.feature_importances_), key=lambda x: x[1] * -1)
print('Feature Importance - Decision Tree (hyperparameter tuning)')
print(tabulate(values_dt, headers, tablefmt="plain"))

Feature Importance - Decision Tree (hyperparameter tuning)
name                                            score
deposit_type_Non Refund                   0.199373
arrival_date_week_number                  0.0816116
market_segment_Online TA                  0.0703453
reservation_status_date_month             0.0681787
reservation_status_date_day               0.0592259
lead_time                                 0.0582863
arrival_date_month                        0.0481972
agent                                     0.0400916
total_of_special_requests                 0.0379441
arrival_date_day                          0.0363977
country_PRT                               0.0313628
required_car_parking_spaces               0.0311332
arrival_date_year                         0.0228228
reservation_status_date_year              0.0220665
reservation_status_day_of_week_Sunday     0.017502
adr                                       0.0160532
previous_cancellations                    0.0150817
stays

### Refine the Random Forest Model (Tuning Parameters)

In [83]:
param_grid_rf = {
    'max_depth':[100,200,None],
    'max_features':['sqrt', 'log2'],
    'n_estimators':[100,500,1000]
}


In [86]:
grid_rf = GridSearchCV(rf, param_grid = param_grid_rf, cv=5, verbose= 1, n_jobs=-1)

grid_rf.fit(X_train, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 48.7min
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed: 100.5min finished


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=47), n_jobs=-1,
             param_grid={'max_depth': [100, 200, None],
                         'max_features': ['sqrt', 'log2'],
                         'n_estimators': [100, 500, 1000]},
             verbose=1)

In [87]:
grid_rf.best_params_

{'max_depth': 100, 'max_features': 'sqrt', 'n_estimators': 1000}

In [48]:
rf2 = RandomForestClassifier(max_depth=100,max_features='sqrt',n_estimators=1000,random_state=47)
rf2.fit(X_train, y_train)
y_pred_rf2 = rf2.predict(X_test)

In [49]:
print (confusion_matrix(y_test, y_pred_rf2))
print (classification_report(y_test, y_pred_rf2))

[[21673   212]
 [ 1416 11785]]
              precision    recall  f1-score   support

           0       0.94      0.99      0.96     21885
           1       0.98      0.89      0.94     13201

    accuracy                           0.95     35086
   macro avg       0.96      0.94      0.95     35086
weighted avg       0.96      0.95      0.95     35086



In [50]:
headers = ["name", "score"]
values_dt = sorted(zip(X_train.columns, rf2.feature_importances_), key=lambda x: x[1] * -1)
print('Feature Importance - Random Forest (hyperparameter tuning)')
print(tabulate(values_dt, headers, tablefmt="plain"))

Feature Importance - Random Forest (hyperparameter tuning)
name                                            score
lead_time                                 0.0682695
deposit_type_Non Refund                   0.0604938
country_PRT                               0.0580207
deposit_type_No Deposit                   0.056613
reservation_status_date_month             0.0554295
arrival_date_week_number                  0.0434318
total_of_special_requests                 0.0400311
adr                                       0.0392294
reservation_status_date_day               0.0372967
arrival_date_day                          0.0326452
arrival_date_month                        0.0324921
agent                                     0.0301249
stays_in_week_nights                      0.0282696
stays_in_weekend_nights                   0.0245819
previous_cancellations                    0.0234087
market_segment_Online TA                  0.0168212
required_car_parking_spaces               0.0166553
rese

In [51]:
# Class imbalance doesn't change the model much 

clf_dt3 = tree.DecisionTreeClassifier(criterion='entropy',max_depth=140, 
                                     min_samples_leaf=1, min_samples_split=3, random_state=47,  class_weight={0:0.4,1:0.6})

clf_dt3.fit(X_train, y_train)
y_pred_dt3 = clf_dt3.predict(X_test)



In [52]:
print (confusion_matrix(y_test, y_pred_dt3))
print (classification_report(y_test, y_pred_dt3))

[[21126   759]
 [  791 12410]]
              precision    recall  f1-score   support

           0       0.96      0.97      0.96     21885
           1       0.94      0.94      0.94     13201

    accuracy                           0.96     35086
   macro avg       0.95      0.95      0.95     35086
weighted avg       0.96      0.96      0.96     35086



In [53]:
# Class imbalance doesn't change the model

rf3 = RandomForestClassifier(max_depth=100,max_features='sqrt',n_estimators=1000,random_state=47, class_weight={0:0.4,1:0.6})
rf3.fit(X_train, y_train)
y_pred_rf3 = rf3.predict(X_test)

In [54]:
print (confusion_matrix(y_test, y_pred_rf3))
print (classification_report(y_test, y_pred_rf3))

[[21660   225]
 [ 1411 11790]]
              precision    recall  f1-score   support

           0       0.94      0.99      0.96     21885
           1       0.98      0.89      0.94     13201

    accuracy                           0.95     35086
   macro avg       0.96      0.94      0.95     35086
weighted avg       0.95      0.95      0.95     35086

